In [1]:
from torchvision import datasets
import torch

data_folder = '~/data/FMNIST'
fmnist = datasets.FashionMNIST(data_folder, download=True,
                              train = True)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting C:\Users\user/data/FMNIST\FashionMNIST\raw\train-images-idx3-ubyte.gz to C:\Users\user/data/FMNIST\FashionMNIST\raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting C:\Users\user/data/FMNIST\FashionMNIST\raw\train-labels-idx1-ubyte.gz to C:\Users\user/data/FMNIST\FashionMNIST\raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting C:\Users\user/data/FMNIST\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to C:\Users\user/data/FMNIST\FashionMNIST\raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting C:\Users\user/data/FMNIST\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to C:\Users\user/data/FMNIST\FashionMNIST\raw



In [21]:
tr_images = fmnist.data
tr_targets = fmnist.targets
unique_values = tr_targets.unique()

In [23]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from torch.utils.data import Dataset, DataLoader
import torch, torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

device = "cuda" if torch.cuda.is_available() else "cpu"
from torchvision import datasets

In [25]:
class FMNISTDataset(Dataset):
    def __init__(self, x, y):
        x = x.float()
        x = x.view(-1, 28*28)
        self.x, self.y = x, y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, ix):
        return self.x[ix].to(device), self.y[ix].to(device)

In [26]:
def get_data():
    train = FMNISTDataset(tr_images, tr_targets)
    trn_dl = DataLoader(train, batch_size= 32, shuffle=True)
    return trn_dl

In [28]:
from torch.optim import SGD
def get_model():
    model = nn.Sequential(
            nn.Linear(28*28, 1000),
            nn.ReLU(),
            nn.Linear(1000, 10)
    ).to(device)
    
    opt = SGD(model.parameters(), lr=0.01)
    loss_fn = nn.CrossEntropyLoss()
    
    return model, loss_fn, opt

In [29]:
def train_batch(x, y, model, opt, loss_fn):
    opt.zero_grad()
    model.train()
    pred = model(x)
    loss = loss_fn(pred, y)
    loss.backward()
    opt.step()
    
    return loss.item()

In [30]:
@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    pred = model(x)
    max_val, argmax = pred.max(-1)
    is_correct = argmax == y
    return is_correct.cpu().numpy().tolist()

In [31]:
train_dl = get_data()
model, loss_fn, optimizer = get_model()

In [33]:
losses, accuracies = [], []

epoch_losses, epoch_accuracies = [], []
    
for epoch in range(5):
    print(epoch)
    epoch_losses, epoch_accuracies = [], []
    for ix, batch in enumerate(iter(train_dl)):
        x, y = batch
        loss = train_batch(x, y, model, optimizer, loss_fn)
        epoch_losses.append(loss)
    
    epoch_loss = np.array(epoch_losses).mean()
    
    for ix, batch in enumerate(iter(train_dl)):
        x, y = batch
        is_correct = accuracy(x, y, model)
        epoch_accuracies.extend(is_correct)
    epoch_accuracy = np.mean(epoch_accuracies)
    
    losses.append(epoch_loss)
    accuracies.append(epoch_accuracy)
        
    

0
1
2
3
4


In [ ]:
epcohs = np.arange(5)+1
plt.figure(figsize=(20, 5))
plt.subplot(121)
plt.title('Loss value over increasing epochs')
plt.plot(epochs, losses, label = 'Training Loss')
plt.legend()

plt.subplot(122)
plt.title('Accuracy value over increasing epochs')
plt.plot(epochs, accuracies, label='Training Accuracy')
